In [123]:
import pandas as pd

import re
import time
import os
from urllib.parse import urlparse

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

In [124]:
df = pd.read_csv('banki.csv')
df

,Bank,Link,Typ,Uwagi
0,Powszechna Kasa Oszczędności Bank Polski SA,https://www.pkobp.pl/oplaty-i-oprocentowanie/o...,Osoby fizyczne,NaN
1,Powszechna Kasa Oszczędności Bank Polski SA,https://www.pkobp.pl/oplaty-i-oprocentowanie/o...,Klienci korporacyjni,NaN
2,Bank Handlowy w Warszawie SA,https://www.citibank.pl/dokumenty/,NaN,[zakładki: klienci indywidualni > konta > tabe...
3,ING Bank Śląski SA,https://www.ing.pl/indywidualni/tabele-i-regul...,Osoby fizyczne,NaN
4,ING Bank Śląski SA,https://www.ing.pl/indywidualni/tabele-i-regul...,Osoby fizyczne,NaN
5,ING Bank Śląski SA,https://www.ing.pl/indywidualni/tabele-i-regul...,Osoby fizyczne,NaN
6,ING Bank Śląski SA,https://www.ing.pl/male-firmy/tabele-i-regulam...,Klienci korporacyjni,NaN
7,ING Bank Śląski SA,https://www.ing.pl/dokumenty-fis-i-korporacji/...,Klienci korporacyjni,NaN
8,Santander Bank Polska SA,https://www.santander.pl/przydatne-informacje/...,Osoby fizyczne,NaN
9,Santander Bank Polska SA,https://www.santander.pl/przydatne-informacje/...,Klienci korporacyjni,NaN


In [165]:
def fix_url(curr, target):
    if target[0] == '/':
        return 'https://' + urlparse(curr).netloc + target
    else:
        return target

def try_download(url, path):
    response = requests.get(url, allow_redirects=True)
    if response.headers.get('content-type') in ['application/pdf', 'application/pdf;charset=UTF-8']:
        open(os.path.join(path, url.rsplit('/', 1)[-1]+'.pdf'), 'wb').write(response.content)
        print('OK', url.rsplit('/', 1)[-1]+'.pdf')
        return True
    else:
        print(':(', url)
        return False
    

In [166]:
search_result = []

for index, row in df[df['Bank'] == 'Bank Handlowy w Warszawie SA'].iterrows():
    response = requests.get(row['Link'])
    soup = BeautifulSoup(response.text, 'html.parser')
    links = soup.find_all('a', href=True)
    search_result.append([row['Bank'], []])
    for link in links:
        try:
            href = link['href']
            html = str(link)
            if re.search('lokat', link['title'], re.IGNORECASE) and re.search('konto', link['title'], re.IGNORECASE):
                url = fix_url(row['Link'], href)
                if try_download(url, '/home/krzysztof/Projekty/Supervisionhack3/scraper/downloads'):
                    search_result[-1][1].append(href)
        except:
            pass

Oferta Lokat Konto Osobiste Citi Priority CitiKonto obowiÄzuje od 18.10.2023
Regulamin oferty âKonto i Lokata Citigoldâ
